# 실습 09: NER 기반 핵심 정보 추출

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/leecks1119/document_ai_lecture/blob/master/notebooks/Lab09_NER정보추출.ipynb)

## 🎯 학습 목표
- NER(Named Entity Recognition) 이해
- 규칙 기반 정보 추출
- 실전 문서에서 핵심 정보 자동 추출

## ⏱️ 소요 시간: 30분
## 📊 난이도: ⭐⭐⭐☆☆


In [ ]:
# 한글 폰트 설치 및 설정 (Colab용)
!apt-get install -y fonts-nanum fonts-nanum-coding fonts-nanum-extra
!fc-cache -fv

# matplotlib 한글 폰트 설정
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
fontprop = fm.FontProperties(fname=font_path, size=10)
plt.rc('font', family=fontprop.get_name())
plt.rc('axes', unicode_minus=False)

print("✅ 한글 폰트 설정 완료!")


In [ ]:
# 환경 설정
!pip install -q git+https://github.com/leecks1119/document_ai_lecture.git


In [ ]:
# UnifiedNERSystem 사용
from docai_course.ner import UnifiedNERSystem

ner = UnifiedNERSystem()

print("✅ NER 시스템 초기화!")
print("\n지원 엔티티 타입:", list(ner.patterns.keys()))


In [ ]:
# 계약서 정보 추출
contract_text = """
용역 계약서

계약일자: 2025년 3월 1일
발주자: 삼성전자 주식회사
담당자: 김철수 부장
연락처: 02-2000-1234
이메일: chulsoo.kim@samsung.com

수주자: (주)테크솔루션
연락처: 031-456-7890
이메일: contact@techsolution.com

계약금액: 50,000,000원
입금계좌: 123-456-789012
"""

print("📄 계약서 정보 추출 중...\n")
entities = ner.rule_based_ner(contract_text)
ner.visualize_entities(contract_text, entities)


In [ ]:
# 구조화된 데이터로 변환 및 저장
import pandas as pd

data = []
for entity in entities:
    data.append({
        '타입': entity['entity'],
        '값': entity['text'],
        '신뢰도': f"{entity['confidence']*100:.0f}%"
    })

df = pd.DataFrame(data)
print("\n📊 추출된 정보:")
print(df.to_string(index=False))

# CSV로 저장
df.to_csv('contract_entities.csv', index=False, encoding='utf-8-sig')
print("\n✅ 저장 완료: contract_entities.csv")
